In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import sqlite3
import datetime

In [2]:
url = 'https://www.theverge.com/'

In [15]:
response = requests.get(url)
content = response.content
content

b'<!DOCTYPE html><html lang="en-US"><head><meta charSet="utf-8"/><meta name="description" content="The Verge is about technology and how it makes us feel. Founded in 2011, we offer our audience everything from breaking news to reviews to award-winning features and investigations, on our site, in video, and in podcasts."/><meta name="twitter:card" content="summary_large_image"/><meta name="twitter:site" content="@verge"/><meta property="fb:app_id" content="549923288395304"/><meta property="og:description" content="The Verge is about technology and how it makes us feel. Founded in 2011, we offer our audience everything from breaking news to reviews to award-winning features and investigations, on our site, in video, and in podcasts."/><meta property="og:image" content="https://cdn.vox-cdn.com/uploads/chorus_asset/file/23966628/the_verge_social_share.png"/><meta property="og:site_name" content="The Verge"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=n

In [4]:
soup = BeautifulSoup(content, 'html.parser')

In [13]:
articles = soup.find_all('article')
articles

[]

In [6]:
csv_file = open(f'{datetime.datetime.now().strftime("%d%m%Y")}_verge.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['id', 'url', 'headline', 'author', 'date'])

29

In [14]:
conn = sqlite3.connect(f'{datetime.datetime.now().strftime("%d%m%Y")}_verge.db')
c = conn.cursor()

In [8]:
c.execute('''CREATE TABLE IF NOT EXISTS articles(id INTEGER PRIMARY KEY, url TEXT, headline TEXT, author TEXT, date TEXT)''')

In [11]:
for index, article in enumerate(articles):
    headline = article.find('h2').text.strip()
    url = article.find('a')['href']
    author = article.find('span', class_='c-byline__author').text.strip()
    date = article.find('time')['datetime']
    
    # Write the data to the CSV file
    csv_writer.writerow([index, url, headline, author, date])
    
    # Check if the article already exists in the database
    c.execute("SELECT id FROM articles WHERE URL = ?", (url,))
    data = c.fetchone()
    if data is None:
        # Insert the data into the SQLite database if it doesn't exist
        c.execute("INSERT INTO articles (id, URL, headline, author, date) VALUES (?, ?, ?, ?, ?)", 
                  (index, url, headline, author, date))

In [12]:
csv_file.close()
conn.commit()
conn.close()

ProgrammingError: Cannot operate on a closed database.